In [391]:
from newspaper import Article
import newspaper
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request as urllib2
import re
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By  # Import the By class
import nltk
from bs4 import BeautifulSoup,NavigableString
from selenium import webdriver
from newspaper import Article
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pandas as pd
import requests
from requests.exceptions import Timeout

from selenium.webdriver.chrome.options import Options
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from queue import Queue
#nltk.download('punkt')
model = SentenceTransformer('intfloat/e5-large')
from langchain.llms import OpenAIChat
from langchain.callbacks import get_openai_callback
import os
from dotenv import load_dotenv
load_dotenv()
llm = OpenAIChat(model_name='gpt-4-turbo-preview')

/home/waleedbutt/.local/lib/python3.10/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [392]:
def extract_Stateroom_info(url):
    response = requests.get(url, timeout=10)  # 10 seconds timeout
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    # Extract dates
    qq = soup.find_all('div', class_=lambda value: value and value.startswith('mod-dpcalendar-upcoming mod-dpcalendar-upcoming-default mod-dpcalendar-upcoming-') if value else False)
    dates = []
    for div in qq:
        date_spans = div.find_all('span', class_='dp-date__start')
        dates.extend([span.text for span in date_spans])

    # Extract ticket links
    links = soup.find_all('a', class_='acf_url btn')
    ticket_links = [tag.get('href') for tag in links if tag.get('href')]

    # Extract names and 'with' parts
    divs = soup.find_all('div', 'zg-col zg-col-6')
    names, with_parts, event_links = [], [], []
    for div in divs:
        a_tag = div.find('a')
        if a_tag and a_tag.text:
            names.append(a_tag.text)
            event_links.append(a_tag.get('href'))

            with_part = ''
            for sibling in a_tag.parent.next_siblings:
                if isinstance(sibling, NavigableString) and sibling.strip():
                    with_part = sibling.strip()
                    break
            with_parts.append(with_part)

    # Assuming the combined_text contains location name and address
    combined_text = ' '.join([div.get_text(separator=" ", strip=True) for div in soup.find_all('div', 'custom')])
    with get_openai_callback() as cb:
        solution=llm(f"""You are a expert location identifier. Given text, you will find the location name and address for a musical event.The output will be in this format.
            Location_Name@Location_Address The location name can even be found from the name of an image link. If not name or address is found give None instead. Like so
            None@192 a2 phasa town\n
            You are ordered to print nothing but what is asked in the given format.
            {combined_text}""")
        print(cb)
    location_name=solution.split('@')[0]
    location_address=solution.split('@')[1]
    # Here you should integrate your method to extract location name and address from combined_text
    # For demonstration, setting placeholder values
    location_name = "Placeholder Location Name"  # Replace this with actual extraction logic
    location_address = "Placeholder Location Address"  # Replace this with actual extraction logic

    # Create DataFrame
    df = pd.DataFrame({
        'loc_Name': [location_name]*len(names),
        'loc_address': [location_address]*len(names),
        'Singer_Name': names,
        'Event_Time': dates,
        'Ticket Price': [None]*len(names),
        'Ticket Link': ticket_links
    })

    return df

In [393]:
sateroom_urls = [
    """https://thestateroompresents.com/the-state-room""",
    """https://thestateroompresents.com/the-commonwealth-room""",
    """https://thestateroompresents.com/deer-valley""",
    """https://thestateroompresents.com/stateroompresents"""
]

In [394]:
response = requests.get('https://www.theurbanloungeslc.com/', timeout=10)  # 10 seconds timeout
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [395]:
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(15)
driver.get('https://www.theurbanloungeslc.com/')
WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.tix__price"))
    )
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
widgets = soup.find_all('div', class_='tix__widget--body tix__item')
location_names = soup.find_all('p', 'tix__venue')
dates = soup.find_all('p', 'tix__date')
times = soup.find_all('p', 'tix__eventtime')
prices = soup.find_all('div', 'tix__price')
Links = soup.find_all('div', class_='tix__widget--footer tix__item')

In [396]:
Presented_by_text = []
titles_text = []
names_text = []

for widget in widgets:
    # Extracting 'Presented By' text
    presented_by = widget.find('h2', class_='tix__title')
    if presented_by:
        Presented_by_text.append(presented_by.text.strip())
    else:
        Presented_by_text.append('')

    # Extracting 'Title' text
    title = widget.find('p', class_='tix__title--headliner')
    if title:
        titles_text.append(title.text.strip())
    else:
        titles_text.append('')

    # Extracting 'Names' text, ensuring to handle cases where there might be no supporting names
    name = widget.find('p', class_='tix__title--support')
    if name:
        names_text.append(name.text.strip())
    else:
        names_text.append('')  # Append an empty string if no 'tix__title--support' is present

# Combine 'Presented By', 'Title', and 'Names' into one column
combined_info = [f"{pb} - {title} - {name}" if name else f"{pb} - {title}" for pb, title, name in zip(Presented_by_text, titles_text, names_text)]

#

In [397]:
ticket_links = []

for link_div in Links:
    a_tag = link_div.find('a', class_='tix__button')
    if a_tag:
        href = a_tag['href']
        ticket_links.append(href) 

In [398]:
location_addresses=[]

In [399]:
for link in ticket_links:
    print(link)
    response = requests.get(link, timeout=10)  # 10 seconds timeout
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    location_addresses.append(soup.find('div','event-venue-address').text+' '+soup.find('span',itemprop='addressLocality').text+' '+soup.find('span',itemprop='addressRegion').text+' '+soup.find('span',itemprop='postalCode').text)

https://www.24tix.com/event/acjpijh3azfrdmmvldgzlk2wyq
https://www.24tix.com/event/paeuc2xaqzfcjlcd5tjiydpeaa
https://www.24tix.com/event/yt3dbqblmngk3fciraers3uxeq
https://www.24tix.com/event/e3mhyseverdktibf4gc4u64v54
https://www.24tix.com/event/ucye2h3oyzb5nj6eg32mulh4ce
https://www.24tix.com/event/qvxeokfoezfkxn5ummc5srvdum
https://www.24tix.com/event/khfhhwzdgreizfrrw4y2bon4py
https://www.24tix.com/event/adlpx34fhfdf5kgggkdw332s2i
https://www.24tix.com/event/sycbagypj5hmfiiobebatk3asa
https://www.24tix.com/event/zs2jjqf6pfcfxaunbkwormmhpa
https://www.24tix.com/event/zv6wncukqndwbdyti45ww5s5by
https://www.24tix.com/event/sat6hnfatran3oqiy5kyhfy6mm
https://www.24tix.com/event/noebwkmr2vcr3dl4w7sl4367ke
https://www.24tix.com/event/g4aqhan2xfdadbueos6nfm2ubi
https://www.24tix.com/event/mpjcn2xbnnfwvg4avyyivg4lku
https://www.24tix.com/event/fh3fnesz7zdn7jmr2cwhv2lkwi
https://www.24tix.com/event/pnuab6rklzfk5osh4l6trt53uq
https://www.24tix.com/event/46tzp7lmrngnvlo6bjzlr7cwha
https://ww

In [401]:
location_names

[<p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class="tix__venue">The Urban Lounge</p>,
 <p class=

In [402]:
location_names_text = [location_name.text.strip() for location_name in location_names]
dates_text = [date.text.strip() for date in dates]
times_text = [time.text.strip() for time in times]
prices_text = [price.text.strip() for price in prices]

In [403]:
df_combined = pd.DataFrame({
    'Name': combined_info,
    'Venue Name': location_names_text,
    'Venue Address':location_addresses,
    'Date': dates_text,
    'Time': times_text,
    'Price': prices_text,
    'Ticket Link': ticket_links
})

In [406]:
df_combined.to_csv('urbanlounge.csv')

In [407]:
df_combined

,Name,Venue Name,Venue Address,Date,Time,Price,Ticket Link
0,"S&S Presents - Silent Planet - Thornhill, John...",The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Jan 31, 2024",Event: 6:00am,$22.00,https://www.24tix.com/event/acjpijh3azfrdmmvld...
1,S&S Presents - Christine Baird - Sean Marshall...,The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 01, 2024",Event: 7:00am,$10.00,https://www.24tix.com/event/paeuc2xaqzfcjlcd5t...
2,"S&S Presents - Die Shiny - Cassette Drift, Sle...",The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 02, 2024",Event: 7:00am,$10.00,https://www.24tix.com/event/yt3dbqblmngk3fcira...
3,"S&S Presents - DUBWISE - Lost City, Roommate, ...",The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 03, 2024",Event: 8:30am,$12.57,https://www.24tix.com/event/e3mhyseverdktibf4g...
4,S&S Presents - SUGAR: THE NU-METAL PARTY,The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 04, 2024",Event: 8:30am,$14.00,https://www.24tix.com/event/ucye2h3oyzb5nj6eg3...
5,S&S and Popscene SLC Presents - Poolside - Und...,The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 06, 2024",Event: 7:00am,$25.00,https://www.24tix.com/event/qvxeokfoezfkxn5umm...
6,"S&S Presents - Twin Tribes - Urban Heat, Vanda...",The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 07, 2024",Event: 7:00am,$22.00,https://www.24tix.com/event/khfhhwzdgreizfrrw4...
7,90.9FM KRCL Presents - Mayer Hawthorne - Hawth...,The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 08, 2024",Event: 7:00am,$30.00,https://www.24tix.com/event/adlpx34fhfdf5kgggk...
8,"S&S Presents - 4PLAY - Batshit'72, Saccharine",The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 09, 2024",Event: 7:00am,$10.00,https://www.24tix.com/event/sycbagypj5hmfiiobe...
9,S&S Presents - Galentine's: Olivia + Taylor Da...,The Urban Lounge,241 South 500 East Salt Lake City UT 84102,"Feb 10, 2024",Event: 8:00am,$18.00,https://www.24tix.com/event/zs2jjqf6pfcfxaunbk...


In [405]:
# dfs = []

# for url in sateroom_urls:
#     try:
#         df = extract_Stateroom_info(url)  # Call your function to get the DataFrame
#         dfs.append(df)
#     except Exception as e:
#         print(f"Error processing {url}: {e}")

# # Concatenate all DataFrames into a final DataFrame
# final_df = pd.concat(dfs, ignore_index=True)